In [41]:
import pandas as pd
from sqlalchemy import create_engine

server = "OOO"
database = "test"
driver = "ODBC Driver 17 for SQL Server"

engine = create_engine(f"mssql+pyodbc://{server}/{database}?Trusted_Connection=yes&driver={driver}")

# conn = engine.connect()

df = pd.DataFrame(data=[['cat',11],['dog',5],['parrot',2]], columns=('animal','age'))
df.to_sql('animal_table', con=engine, if_exists='replace')

query = "SELECT * FROM animal_table"
df = pd.read_sql(query, con=engine)
df

,index,animal,age
0,0,cat,11
1,1,dog,5
2,2,parrot,2


In [35]:
query = "SELECT * FROM animal_table"
engine = create_engine("mssql+pyodbc://OOO/test?Trusted_Connection=yes&driver=ODBC Driver 17 for SQL Server")
with engine.connect() as connection:
        print(f"Successfully connected to everyloop!")
        result = connection.execute(text(query))
        for row in result:
                print(f"{row.animal} {row.age}")

Successfully connected to everyloop!
cat 11
dog 5
parrot 2


In [ ]:
with engine.connect() as conn: 
	result = conn.execute(query)
	for row in result:
		print(f"{row.animal} {row.age}")

In [30]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine("mssql+pyodbc://OOO/test?Trusted_Connection=yes&driver=ODBC Driver 17 for SQL Server")


df = pd.DataFrame(data=[['cat',11],['dog',5],['parrot',2]], columns=('animal','age'))
df.to_sql('animal_table', con=engine, if_exists='replace')

query = "SELECT * FROM animal_table"

with engine.connect() as conn: 
	result = conn.execute(text(query))
	for row in result:
		print(f"{row.animal} {row.age}")


cat 11
dog 5
parrot 2


In [ ]:
from sqlalchemy import create_engine

server = "OOO"
database = "everyloop"
driver = "ODBC Driver 17 for SQL Server"

engine = create_engine(f"mssql+pyodbc://{server}/{database}?Trusted_Connection=yes&driver={driver}")

with engine.connect() as connection:
 print(f"Successfully connected to databse {database} on server {server}!")
 

 

Successfully connected to databse everyloop on server OOO!


In [ ]:
from sqlalchemy import create_engine, select, MetaData, Table, Column, Integer, String

server = "OOO"
database = "everyloop"
driver = "ODBC Driver 17 for SQL Server"

engine = create_engine(f"mssql+pyodbc://{server}/{database}?Trusted_Connection=yes&driver={driver}")

# Create a Table
meta = MetaData()
animal_table = Table(
    'animal_table', meta, 
    Column('id', Integer, primary_key = True),
    Column('animal', String),
    Column('age', Integer)
)
meta.drop_all(engine, checkfirst=True)
meta.create_all(engine)

# Insert Values into the Table
with engine.connect() as connection:
    connection.execute(
        animal_table.insert().values([
            {'id':1, 'animal': 'cat', 'age': 3},
            {'id':2, 'animal': 'dog', 'age': 5}
        ])
    )
    connection.commit()

# Two Ways of Retrieving the Information from the Table 
with engine.connect() as connection:
    # Using a raw SQL query with text()
    result = connection.execute(text("SELECT id, animal, age FROM animal_table WHERE age > :age"), {"age": 2})
    for row in result:
        print(f"id: {row.id}, animal: {row.animal}, age: {row.age}")

    # Alternatively, using SQLAlchemy's Select statement (more ORM-like)
    from sqlalchemy import select
    stmt = select(animal_table.c.id, animal_table.c.animal, animal_table.c.age).where(animal_table.c.age > 2)
    result = connection.execute(stmt)
    for row in result:
        print(f"id: {row.id}, animal: {row.animal}, age: {row.age}")


id: 1, animal: cat, age: 3
id: 2, animal: dog, age: 5
id: 1, animal: cat, age: 3
id: 2, animal: dog, age: 5
